In [1]:
from manim import *
import math
from variables import *

import sys
sys.path.append("..") 
from manim_functions import *

/opt/conda/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [23]:
var = "25-Decoder_Normalization"

!manim -ql -v WARNING  {var}.py generate

from IPython.display import Video
Video(f"media/videos/{var}/480p15/generate.mp4", embed=True)

/opt/conda/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
Manim Community v0.19.0

                                                                                

In [24]:
var = "notes_25"

!manim -ql -v WARNING  {var}.py generate

from IPython.display import Video
Video(f"media/videos/{var}/480p15/generate.mp4", embed=True)

/opt/conda/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
Manim Community v0.19.0

                                                                                